Code that clusters ballot graph based on comparing PL probability of generated ballots from intervals. (BT needs further theorizing to deal with partial ballots, truncation.)

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from pref_schedule import pref_schedule
import random 

In [7]:
ballot_dict = {(1,2,3):400, (1, 3, 2): 100, (2, 1, 3): 150, (2, 3, 1): 50, (3,2,1):400, (3, 1, 2): 300, (3,):5, (2,):7, (1,):9}

p = pref_schedule(ballot_dict)

In [26]:
G = pref_schedule.build_graph(3)

In [10]:
class prefInterval:
    def __init__(self, support_interval, trunc_interval):
        self.support_interval = support_interval
        self.trunc_interval = trunc_interval

    def get_support_interval(self):
        return self.support_interval
    
    def get_trunc_interval(self):
        return self.trunc_interval

In [11]:
support_interval = {1: 0.5, 2: 0.4, 3: 0.1}
trunc_interval = [0, 0.5, 0.7]

int = prefInterval(support_interval, trunc_interval)
int.get_support_interval()


{1: 0.5, 2: 0.4, 3: 0.1}

In [17]:
def pl_probability(ballot, pref_interval):
    '''
    Computes PL probability of a ballot given an interval
    TODO: adapt for truncation? 
    '''
    prob = 1

    remaining_interval_length = sum([i for i in pref_interval.get_support_interval().values()]) # should be 1 
    
    for cand in ballot: 
        # multiply prob by the length of the corresponding sub-interval, divided by the length
        # of the remainin subinterval 
        prob = prob * (pref_interval.get_support_interval()[cand] / remaining_interval_length)
        # update the length of the remaining subinterval by subtracting the length of the 
        # subinterval just used 
        remaining_interval_length -= pref_interval.get_support_interval()[cand]
    
    return prob


In [13]:
def pl_probability_with_truncation(ballot, pref_interval):
    '''
    Computes PL probability of a ballot given an interval with candidate strenghs and another interval with probabilities of truncation at 
    each stage. trunc_interval[0] should be 0 if we don't allow empty ballots  
    '''
    prob = 1

    remaining_interval_length = sum([i for i in pref_interval.get_support_interval().values()]) # should be 1 
    
    for i, cand in enumerate(ballot): 
        # no truncation at stage i
        prob = prob * (1 - pref_interval.get_trunc_interval()[i])
        # multiply prob by the length of the corresponding sub-interval, divided by the length
        # of the remainin subinterval 
        prob = prob * (pref_interval.get_support_interval()[cand] / remaining_interval_length)

        # update the length of the remaining subinterval by subtracting the length of the 
        # subinterval just used 
        remaining_interval_length -= pref_interval.get_support_interval()[cand]
    
    # if there is truncation, account for this probability 
    if len(ballot) < len(trunc_interval):
        prob = prob * pref_interval.get_trunc_interval()[len(ballot)]

    return prob

In [14]:
def bt_probability(ballot, pref_interval):
    # note -- technically the bt_probability is only prorportional to the given value. however, for purposes of classificaiton, 
    # this shouldn't matter. 
    # TODO: How do we deal with truncation? (Same as PL?) What about ballots of length 1? (prob proportional to length of sub-interval of ranked candidate?
    # This would seem to unfairly penalized bullet votes as compared to votes of length 2)
    # How do we factor in diminishing consideration down the ballot? 

    prob = 1
    for i in range(len(ballot)):
        support_for_i = pref_interval.get_support_interval()[ballot[i]]
        for j in range(i+1, len(ballot)):
            support_for_j = pref_interval.get_support_interval()[ballot[j]]
            prob = prob * (support_for_i / (support_for_i + support_for_j))

    return prob 

In [ ]:
print(pl_probability((1,), int))

0.5


In [ ]:
print(pl_probability_with_truncation((2,), int))

0.2


In [19]:
print(bt_probability((1, 2, 3), int))

0.37037037037037046


In [24]:
def compute_clusters_from_intervals(pref_intervals, graph, prob_funciton, ballot_dict): 
    cluster_map = {}
    fit_score = 0

    for ballot in graph.nodes():
        probs = [prob_funciton(ballot, i) for i in pref_intervals]
        max_prob = max(probs)
        best_intervals = [pref_intervals[i] for i, prob in enumerate(probs) if prob == max_prob]
        chosen_interval = random.choice(best_intervals)
        
        cluster_map[ballot] = chosen_interval.get_support_interval()
        fit_score += max_prob * ballot_dict[ballot]
    
    num_ballots = sum([count for count in ballot_dict.values()]) 
    fit_score = fit_score / num_ballots

    return cluster_map, fit_score

In [27]:
pref_intervals = [prefInterval({1: 0.5, 2: 0.3, 3: 0.1}, [0, 0.5, 0.7]), prefInterval({1: 0.1, 2: 0.4, 3: 0.5}, [0, 0.5, 0.7])]
compute_clusters_from_intervals(pref_intervals, G, pl_probability_with_truncation, ballot_dict)

({(1,): {1: 0.5, 2: 0.3, 3: 0.1},
  (1, 2, 3): {1: 0.5, 2: 0.3, 3: 0.1},
  (1, 3, 2): {1: 0.5, 2: 0.3, 3: 0.1},
  (2,): {1: 0.1, 2: 0.4, 3: 0.5},
  (2, 3, 1): {1: 0.1, 2: 0.4, 3: 0.5},
  (2, 1, 3): {1: 0.5, 2: 0.3, 3: 0.1},
  (3,): {1: 0.1, 2: 0.4, 3: 0.5},
  (3, 1, 2): {1: 0.1, 2: 0.4, 3: 0.5},
  (3, 2, 1): {1: 0.1, 2: 0.4, 3: 0.5}},
 0.048897490030494956)

In [28]:
compute_clusters_from_intervals(pref_intervals, G, bt_probability, ballot_dict)

({(1,): {1: 0.1, 2: 0.4, 3: 0.5},
  (1, 2, 3): {1: 0.5, 2: 0.3, 3: 0.1},
  (1, 3, 2): {1: 0.5, 2: 0.3, 3: 0.1},
  (2,): {1: 0.5, 2: 0.3, 3: 0.1},
  (2, 3, 1): {1: 0.1, 2: 0.4, 3: 0.5},
  (2, 1, 3): {1: 0.5, 2: 0.3, 3: 0.1},
  (3,): {1: 0.1, 2: 0.4, 3: 0.5},
  (3, 1, 2): {1: 0.1, 2: 0.4, 3: 0.5},
  (3, 2, 1): {1: 0.1, 2: 0.4, 3: 0.5}},
 0.2928696861886517)

In [ ]:

# for edge in G.edges():
#     u, v = edge 
#     # find the discrepancy between the two nodes (swap or truncation) 
#     if len(u) == len(v):
    
#     # based on the discrepancy, calculate the corredsponding edge weight from the interval

#     # add the edge back to the graph with the appropriate weight 
#     G.add_edge(u, v, weight = 2)



[((1,), (1, 2, 3)), ((1,), (1, 3, 2)), ((1, 2, 3), (1, 3, 2)), ((1, 2, 3), (2, 1, 3)), ((1, 3, 2), (3, 1, 2)), ((2,), (2, 3, 1)), ((2,), (2, 1, 3)), ((2, 3, 1), (2, 1, 3)), ((2, 3, 1), (3, 2, 1)), ((3,), (3, 1, 2)), ((3,), (3, 2, 1)), ((3, 1, 2), (3, 2, 1))]
